# Interactive Time Series Visualization Assignment

In [1]:
import pandas as pd
import plotly_express as px
from ipywidgets import interact

In [2]:
def ihistogram(df, field, bins=None, title=""):
    fig = px.histogram(df, x=field, title=title, nbins=bins, template="none").update(
        layout=dict(title=dict(x=0.5))
    )

    fig.update_yaxes(title_text="Number of Records")
    fig.update_traces(
        marker_color="lightskyblue", marker_line_color="black", marker_line_width=1
    )

    fig.show()

### Load the Online Retail.xlsx Excel file into a Pandas dataframe.

In [3]:
df = pd.read_excel("data/Online Retail.xlsx")

In [4]:
df.head(2)

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom


In [5]:
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
df["date"] = df["InvoiceDate"].dt.date

### Create an interactive histogram to explore the distributions of daily (by date) Quantity, UnitPrice, and Revenue. Include a dropdown widget where you can switch between the three fields.

**Hint:** You will need to create a Date field and then aggregate on it. The Quantity and Revenue fields should be summed and the UnitPrice field should be averaged (mean).

In [6]:
quant_agg = df.groupby("date").sum()["Quantity"].reset_index()
rev_agg = df.groupby("date").sum()["Revenue"].reset_index()
price_agg = df.groupby("date").mean()["UnitPrice"].reset_index()

In [7]:
rev_agg

,date,Revenue
0,2010-12-01,46219.29
1,2010-12-02,47283.53
2,2010-12-03,23576.01
3,2010-12-05,31315.64
4,2010-12-06,31014.21
...,...,...
300,2011-12-05,55920.60
301,2011-12-06,45584.19
302,2011-12-07,68699.21
303,2011-12-08,50214.15


In [8]:
fields = ["Quantity", "UnitPrice", "Revenue"]
fields_dict = {"Quantity": quant_agg, "UnitPrice": price_agg, "Revenue": rev_agg}


@interact(field=fields)
def viz(field):
    px_df = fields_dict[field]
    fig = px.histogram(px_df, x="date", y=field, nbins=30)
    fig.show()

interactive(children=(Dropdown(description='field', options=('Quantity', 'UnitPrice', 'Revenue'), value='Quant…

### Create an interactive line chart where you can explore Quantity and Revenue by day (date) for each country. Include two dropdown widgets - one to choose between the Quantity and Revenue fields and the other to choose the Country.

In [9]:
countries = sorted(df["Country"].unique())
line_fields = ["Quantity", "Revenue"]


@interact(country=countries, field=line_fields)
def viz(country, field):
    country_df = df[df["Country"] == country]
    agg_df = country_df.groupby("date").sum()[field].reset_index()
    fig = px.line(agg_df, x="date", y=field)
    fig.show()

interactive(children=(Dropdown(description='country', options=('Australia', 'Austria', 'Bahrain', 'Belgium', '…

### Create an interactive scatter plot showing the relationships between daily Quantity, UnitPrice, and Revenue for the United Kingdom. Include two dropdown boxes that let you choose between the 3 fields - one for the x axis and one for the y axis of your scatter plot. Size the data points according to Revenue.

In [11]:
uk_agg_df

,date,Quantity,Revenue,UnitPrice
0,2010-12-01,21308,42213.58,3.172830
1,2010-12-02,30987,47123.68,3.187544
2,2010-12-03,7646,18833.99,3.143373
3,2010-12-05,13603,25807.89,2.818036
4,2010-12-06,15515,29651.87,2.875897
...,...,...,...,...
300,2011-12-05,35874,52893.63,2.571737
301,2011-12-06,25909,42063.55,2.249006
302,2011-12-07,29078,49238.47,2.328416
303,2011-12-08,25441,47146.11,2.565525


In [15]:
fields = ["Quantity", "UnitPrice", "Revenue"]
uk = df[df["Country"] == "United Kingdom"]
uk_agg_df = (
    uk.groupby("date")
    .agg({"Quantity": "sum", "Revenue": "sum", "UnitPrice": "mean"})
    .reset_index()
)


@interact(field1=fields, field2=fields)
def viz(field1, field2='Revenue'):
    fig = px.scatter(uk_agg_df, x=field1, y=field2, size="Revenue", hover_name='date')
    fig.show()

interactive(children=(Dropdown(description='field1', options=('Quantity', 'UnitPrice', 'Revenue'), value='Quan…

### Create a bar chart showing the top X products sold (by quantity) in the United Kingdom in a specific month. Use a dropdown box to select the month name and a slider to show the top X products. The range for X should be from 5 to 25. Make sure the bars are sorted in descending order according to their total quantity sold for the month.

Bonus points if you can figure out how to ensure that the month names in the dropdown appear in their correct order (January, February, March, April, etc.).

In [23]:
df['month'] = df['InvoiceDate'].dt.month_name()
df['month_num'] = df['InvoiceDate'].dt.month
df.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country,date,month,month_num
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom,2010-12-01,December,12
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom,2010-12-01,December,12
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom,2010-12-01,December,12
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.2,17511,United Kingdom,2010-12-01,December,12
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13408,United Kingdom,2010-12-01,December,12


In [24]:
aggregated = df.groupby(['month', 'month_num', 'Description']).agg({"Quantity": "sum", "Revenue": "sum", "UnitPrice": "mean"}).reset_index()

In [28]:
aggregated = aggregated.sort_values('month_num')

In [31]:
def ibarchart(df, x, y, order=None, title=''):
    fig = px.bar(df, x=x, y=y, title=title, template='none')

    fig.update_traces(marker_color='lightskyblue',
                      marker_line_color='black', 
                      marker_line_width=1)
    
    fig.update_layout(xaxis={'categoryorder':'array', 
                             'categoryarray': order})
    
    fig.show()

In [33]:
months = list(aggregated['month'].unique())

@interact(Month = months, Top=(5,25))

def viz(Month, Top):
    monthly = aggregated[aggregated['month']==Month]
    top_prod = monthly.sort_values('Quantity', ascending=False).head(Top)
    ibarchart(top_prod, 'Description', 'Quantity', title=f'Top {Top} Products Sold in {Month}')

interactive(children=(Dropdown(description='Month', options=('January', 'February', 'March', 'April', 'May', '…